In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import sys

from torch import nn

### control line

In [2]:
# csv_file = '../data/img_conv2.csv'
random_seed = 42
# dataset_path = "../data/HAM10000/images/"
# groundtruth_file = '../data/HAM10000/GroundTruth.csv'
# feature_vector_file_path = '../data/HAM10000/img_feature_no_masked.csv'

device = "cuda" if torch.cuda.is_available() else "cpu"

# model_select_signal = 'resnet18'
model_select_signal = 'vgg16'

data_select_signal = 'skin'
# data_select_signal = 'chest CT'

## dataset prepocessing

In [5]:
from helperFunction.CustomImageDataset import CustomImageDataset
import os
import random
import torchvision
import json

# 數據轉換
transform_std = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform224 = torchvision.transforms.Compose([
    transforms.Resize(size=(224,224)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])

transform = transform_std


# all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# random.seed(42)

# num_total_samples = len(all_files)
# split_ratio = 0.8  # 80% 的数据用于训练，20% 用于测试

# # 随机打乱数据集
# random.shuffle(all_files)

# # 计算分割点
# split_idx = int(num_total_samples * split_ratio)

# # 分割数据集
# train_files = all_files[:split_idx]
# test_files = all_files[split_idx:]

# 載入train & test file list
# with open('test_files_list.json', 'r') as f:
#      test_files = json.load(f)
# with open('train_files_list.json', 'r') as f:
#      train_files = json.load(f)


# # 加載數據
# train_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# test_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
# @ transfrom test unit

# import unittest
# import numpy as np
# import torch
# from torchvision import transforms
# from PIL import Image

# class TestTransforms224Gray32bit(unittest.TestCase):
#     def setUp(self, img_path=None):
#         # 创建一个纯白色的测试图像，尺寸为 300x300
#         if img_path :
#             self.image = Image.open(img_path).convert('L')
#         else :
#             self.image = Image.new('L', (300, 300), color='white')

#     def test_transform_flow(self):
#         transform224_gray_32bit = transforms.Compose([
#             transforms.Lambda(lambda img: np.array(img).astype(np.float32) / 255.0),
#             transforms.Lambda(lambda x: torch.from_numpy(x)),
#             transforms.Resize(256),
#             transforms.CenterCrop(224),
#             transforms.Normalize(mean=[0.485], std=[0.229]),
#         ])
        
#         # 应用转换流程
#         transformed_img = transform224_gray_32bit(self.image)
        
#         # 检查转换后的类型和形状
#         self.assertTrue(isinstance(transformed_img, torch.Tensor), "Output should be a Torch Tensor")
#         self.assertEqual(transformed_img.dtype, torch.float32, "Output tensor should have dtype float32")
#         self.assertEqual(transformed_img.size(), (1, 224, 224), "Output tensor should have shape (1, 224, 224)")
        
#         # 检查值的范围是否合理（因为原图是纯白的，所以归一化后应该有一个固定的范围）
#         expected_value = (1 - 0.485) / 0.229
#         self.assertTrue(torch.allclose(transformed_img.mean(), torch.tensor(expected_value), atol=1e-5),
#                         "Normalized values do not match expected range")

# test = TestTransforms224Gray32bit()
# test.setUp('../data/chestCTData/images/train/adenocarcinoma/000000 (6).png')

# test.test_transform_flow()

In [7]:
sys.path.append(os.path.abspath('..'))


from data.HAM10000.ham10000Dataloader import HAM10000DataProcessor
from data.chestCTData.chestCTDataloader import ChestCTDataProcessor

if data_select_signal == 'skin':
    dataContainer = HAM10000DataProcessor(transform=transform_std)
elif data_select_signal == 'chest CT':
    dataContainer = ChestCTDataProcessor(transform=transform_std)

train_dataloader , test_dataloader = dataContainer.returnDataloaders()
train_files , test_files = dataContainer.returnDatasetFilenames()
feature_vector_file_path = dataContainer.returnFeatureVectorFilename()

num_classes = dataContainer.getNumClasses()



In [9]:
print(type(transform224))

# print(train_files[0:5])

<class 'torchvision.transforms.transforms.Compose'>


### load color feature data base on dataloader filename idx

In [10]:
# from helperFunction.XgbHelperFunction import csvkeylistToData


# train_feature_vectors = csvkeylistToData(feature_vector_file_path, train_files)
# test_feature_vectors = csvkeylistToData(feature_vector_file_path, test_files)

### define XGB eval recorder function

In [11]:
# define XGB training function
from helperFunction.helperFunctions import dataloaderToFeatureData , calImgFeatureVector
from helperFunction.XgbHelperFunction import  train_predict, calBestIterOfXGB
import xgboost as xgb

 
def recordXGBoutput(model:nn.Sequential, train_dataloader, test_dataloader, enable_muti_module=False):
  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)

  if enable_muti_module:
  #   for idx , (data, label) in enumerate(train_dataloader):
  #     train_features[idx] = np.concatenate(train_features[idx] , calImgFeatureVector(data))
  #   for idx , (data, label) in enumerate(test_dataloader):
  #     test_features[idx] = np.concatenate(test_features[idx] , calImgFeatureVector(data))

    for idx,feature in enumerate(test_features):
        feature = np.concatenate((feature , np.array(test_feature_vectors[idx])))
    for idx,feature in enumerate(train_features):
        feature = np.concatenate((feature , np.array(train_feature_vectors[idx])))

  xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=num_classes)

  iter ,f1 ,acc = calBestIterOfXGB(train_features, train_labels, test_features, test_labels, enable_f1_metric=True)

  # f1, acc = train_predict(xgb_model, train_features, train_labels,  test_features, test_labels)

  print("======eval finish!=========")

  return f1, acc, iter
  


### define ML eval recorder function

In [12]:
def recordMLoutput(ML_model, model:nn.Sequential, train_dataloader, test_dataloader, enable_muti_module=False):
  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)

  if enable_muti_module:
    for idx,feature in enumerate(test_features):
        feature = np.concatenate((feature , np.array(test_feature_vectors[idx])))
    for idx,feature in enumerate(train_features):
        feature = np.concatenate((feature , np.array(train_feature_vectors[idx])))



  f1, acc = train_predict(ML_model, train_features, train_labels,  test_features, test_labels)

  print("======eval finish!=========")

  return f1, acc 

## init nn module

### 初始化模型

In [16]:
from torchinfo import summary

if data_select_signal == 'skin':
    model_folder_path = "../model/HAM10000"
elif data_select_signal == 'chest CT':
    model_folder_path = "../model/CT chest"



# resnet101 = models.resnet101(pretrained=True)
# #  ===================================
# # 加載預訓練的ResNet模型
# resnet18 = models.resnet18(pretrained=True)
# resnet18 = torch.nn.Sequential(*(list(resnet18.children())[:-1]))  # 移除最後的全連接層

#  ===================================
# 加載訓練好的ResNet模型
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, num_classes)
resnet18.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet18.pth"))
# resnet18_7  = torch.nn.Sequential(*(list(resnet18_7.children())[:-1]))  # 移除最後的全連接層

# #  =========================
# resnet50 = models.resnet50(pretrained=True)
# num_ftrs = resnet50.fc.in_features
# resnet50.fc = nn.Linear(num_ftrs, num_classes)
# resnet50.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet50_7.pth"))

# #  ===================================
# # 加載預訓練的VGG模型
# vgg16 = models.vgg16(pretrained=True)
# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層

#  ===================================
# 載入訓練好的vgg
vgg16 = models.vgg16(pretrained=True)
classifier = list(vgg16.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 輸出的種類數
classifier.append(torch.nn.Linear(4096, num_classes))

# 替换原始模型的分类器
vgg16.classifier = torch.nn.Sequential(*classifier)

vgg16.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_VGG16.pth"))

# 使用nn.Sequential的方式取代torch.flatten的功能
new_classfier = nn.Sequential(
    nn.Flatten(),
    vgg16.classifier,
)

vgg16.classifier = new_classfier

summary(vgg16, [1,3,224,224])
# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 7]                    --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

In [17]:
# 決定使用的模型

if model_select_signal == 'resnet18':
    model_0 = resnet18
elif model_select_signal == 'vgg16':
    model_0 = vgg16

model_0 = model_0.to(device)

summary(model_0, input_size=[1,3,224,224])

# get var name
model_0_name = [name for name, val in globals().items() if val == model_0][0]

In [18]:
# @ function test unit : dataloaderToFeatureData w/ img feature enhence
train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)

print(len(train_features))
print(type(train_features))
print(train_features[0])

feature size is :8012
8012
<class 'numpy.ndarray'>
[ 2.3175943   0.92064023  0.68780285  1.7439535  -1.1743599  -1.2689987
 -1.984735  ]


In [ ]:
# @ unit test : find specific layers

# import torchvision.models as models

# # 加载预训练的ResNet18模型
# model_0 = models.resnet18(pretrained=True)

# # 初始化层计数器
# total_layers = 0

# # 遍历模型的所有子模块和层
# for name, layer in model_0.named_modules():
#     # 打印每一层的名称和它的具体类型，这一步是可选的，但对理解模型结构很有帮助
#     # print(name, layer.__class__.__name__)

#     # 对所有层进行计数（包括卷积层、全连接层等）
#     # 如果只想计算特定类型的层（如卷积层Conv2d），则需要添加判断条件
#     total_layers += 1

# # 打印总层数
# print(f'Total number of layers: {total_layers}')

# # 示例：仅计算Conv2d层的数量
# conv_layers = 0
# for name, layer in model_0.named_modules():
#     if isinstance(layer, torch.nn.Sequential):
#         print(name, layer.__class__.__name__)
#         conv_layers += 1

# print(f'Total number of Sequential layers: {conv_layers}')

### 建立輸出為不同隱藏層的model

In [19]:
from helperFunction.helperFunctions import createDetailLayerVersions 
# detail version

list_of_models = createDetailLayerVersions(model_0)

# block level version

# list_of_models = []

# layer = 10
# list_of_models.append((model_0 , "layer:"+str(layer)))
# model = model_0

# while layer > 0:
#     model =  torch.nn.Sequential(*(list(model.children())[:-1])) 
#     layer -= 1
#     list_of_models.append((model , "layer:"+str(layer)))

# print(list_of_models)

# len(list(model.children()))

總層數為: 44層


In [ ]:
# @ unit test : eval ability of model, in list of models
# summary(model_0, input_size=[1,3,224,224])
# summary(list_of_models[67][0] , input_size=[1,3,224,224])
# summary(list_of_models[66][0] , input_size=[1,3,224,224])
# summary(list_of_models[0][0] , input_size=[1,3,224,224])

In [ ]:
# @ unit test : dataloaerToFeatureData()

# # model.forward( {data in datalaoder} ) -> (features , label)
# from helperFunction.helperFunctions import dataloaderToFeatureData

# model_0 = torch.nn.Sequential(*(list(model_0.children())[:-1])) 

# test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)



In [20]:
# @ model test : 確認載入的model性能與原本相符

from helperFunction.TrainHelper import TrainingHelper

train_helper = TrainingHelper(model_0,
                              train_dataloader=train_dataloader,
                              test_dataloader=test_dataloader,
                              loss_fn=nn.CrossEntropyLoss(),
                              optimizer=torch.optim.SGD(model_0.parameters(), lr = 0.001),
                              device=device)

print(train_helper.test_step())


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(0.968170813624821, 0.6598266499582288, 0.6505298027279679)


In [ ]:
# from helperFunction.helperFunctions import calDetailModelLayersNum


# summary(list_of_models[23][0], input_size=[1,3,224,224],depth=4)

# for i in range(calDetailModelLayersNum(model_0)):
#     try : 
#         summary(list_of_models[i][0], input_size=[1,3,224,224],depth=4)
#     except:
#         print("mat error : {} layer".format(i))

In [ ]:
# @ unit test : model versions eval ability
summary(list_of_models[2][0], input_size=[1,3,224,224])
# print(list_of_models[1][0])
# print(list_of_models[0][0])
# type(list_of_models[0][0])
# type(list_of_models[1][0])

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 512, 7, 7]            --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [ ]:
# @ testing : CNN model motification
# from other_program.custom_model.VGG16 import VGG16
# import inspect

# model_0dot0 = VGG16()

# list_of_dummy_model = createDetailLayerVersions(model_0dot0)

# summary(list_of_dummy_model[1][0], [1,3,224,224])

# for i in range(calDetailModelLayersNum(model_0dot0)):
#     try : 
#         summary(list_of_dummy_model[i][0], input_size=[1,3,224,224],depth=4)
#     except:
#         print("mat error : {} layer".format(i))

### 記錄下不同layer的輸出接到xgb的結果

In [21]:
from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

XGB_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "iters":[]}

layer_cnt = 0

for model, model_name in list_of_models:
    input_size = ""
    # 獲取model的output size
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 224, 224]).to(device))
        except RuntimeError:
            print("mat mismatch!!")
            continue
        
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))
    try:
        f1, acc, iter = recordXGBoutput(model, train_dataloader, test_dataloader,enable_muti_module=False)
    except MemoryError:
        print("Not enough memory!! escape form loop")
        break
    except RuntimeError:
        print("mat mismatch!!")
        continue
    print("best acc is : {}".format(acc[iter]))
    print("best f1 is : {}".format(f1[iter]))
    print("model_name : {}".format(model_name) )
    # cal output size
    
    XGB_res["model_name"].append(model_name)
    XGB_res["output_size"].append(input_size)
    XGB_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])
    cur_acc = acc[iter]
    XGB_res["acc"].append((-cur_acc) if cur_acc < 0 else cur_acc)
    cur_f1 = f1[iter]
    XGB_res["f1"].append((-cur_f1) if cur_f1 < 0 else cur_f1)
    XGB_res["iters"].append(iter)

    layer_cnt += 1
    if layer_cnt >= 12:
        break



# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(XGB_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = "XGB_model_results_" + model_0_name + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed
    

input size is :[1, 7]
cal CNN model output...
feature size is :2003
feature size is :8012
Best iteration: 32
======eval finish!=========
best acc is : -0.718922
best f1 is : -0.689144
model_name : layer:44
input size is :[1, 4096]
cal CNN model output...
feature size is :2003
feature size is :8012
Best iteration: 50
======eval finish!=========
best acc is : -0.787319
best f1 is : -0.764532
model_name : layer:43
input size is :[1, 4096]
cal CNN model output...
feature size is :2003
feature size is :8012
Best iteration: 50
======eval finish!=========
best acc is : -0.787319
best f1 is : -0.764532
model_name : layer:42
input size is :[1, 4096]
cal CNN model output...
feature size is :2003
feature size is :8012
Best iteration: 38
======eval finish!=========
best acc is : -0.784823
best f1 is : -0.764504
model_name : layer:41
input size is :[1, 4096]
cal CNN model output...
feature size is :2003
feature size is :8012
Best iteration: 64
======eval finish!=========
best acc is : -0.787319
bes

XGBoostError: [20:08:57] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\io.h:232: bad_malloc: Failed to allocate 13317808704 bytes.

### 紀錄其他ML模型的結果

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

# clf setting 

# csv_model_name , clf = "Linear" , LogisticRegression()
# csv_model_name , clf = "SVC" , SVC(kernel='rbf',gamma='auto', probability=True)
csv_model_name , clf = "RandomForest" , RandomForestClassifier(n_estimators=100)
# csv_model_name , clf = "XGB" , xgb.XGBClassifier()



linear_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "eval_time":[]}


layer_cnt = 0

for model, model_name in list_of_models:
    input_size = ""
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 244, 244]).to(device))
        except:
            print("MAT ERROR!!!")
            continue
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))

    f1, acc = recordMLoutput(clf, model, train_dataloader, test_dataloader,enable_muti_module=False)


    # cal output size
    
    linear_res["model_name"].append(model_name)
    linear_res["output_size"].append(input_size)
    linear_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])
    
    linear_res["acc"].append(acc)
    linear_res["f1"].append(f1)

    layer_cnt += 1
    if layer_cnt >= 15:
        break

    print(model_name)



# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(linear_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = csv_model_name + "_model_results_" + model_0_name + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed

input size is :[1, 4]
cal CNN model output...
feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。
訓練時間 0.0750 秒
預測時間 in 0.0010 秒
預測時間 in 0.0000 秒
訓練集的 F1 score和acc分別為: 0.9967 , 0.9967。
測試集的 F1 score和acc分別為: 0.8164 , 0.8159。
======eval finish!=========
layer:68
input size is :[1, 512, 1, 1]
cal CNN model output...
feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。
訓練時間 0.1623 秒
預測時間 in 0.0020 秒
預測時間 in 0.0010 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.7999 , 0.8000。
======eval finish!=========
layer:67
input size is :[1, 512, 8, 8]
cal CNN model output...


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


訓練時間 6.1888 秒
預測時間 in 0.0530 秒
預測時間 in 0.0280 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.7907 , 0.7905。
======eval finish!=========
layer:66
input size is :[1, 512, 8, 8]
cal CNN model output...
feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。
訓練時間 3.7406 秒
預測時間 in 0.0530 秒
預測時間 in 0.0340 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.8018 , 0.8032。
======eval finish!=========
layer:65
input size is :[1, 512, 8, 8]
cal CNN model output...


...

訓練時間 4.8932 秒
預測時間 in 0.0530 秒
預測時間 in 0.0280 秒
訓練集的 F1 score和acc分別為: 0.9984 , 0.9984。
測試集的 F1 score和acc分別為: 0.7629 , 0.7619。
======eval finish!=========
layer:55
input size is :[1, 512, 8, 8]
cal CNN model output...
feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


訓練時間 5.1591 秒
預測時間 in 0.0662 秒
預測時間 in 0.0420 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.7716 , 0.7746。
======eval finish!=========
layer:54
input size is :[1, 512, 8, 8]
cal CNN model output...
feature size is :315
feature size is :613
訓練 LogisticRegression 模型，樣本數: 613。
訓練時間 4.9958 秒
預測時間 in 0.0600 秒
預測時間 in 0.0280 秒
訓練集的 F1 score和acc分別為: 1.0000 , 1.0000。
測試集的 F1 score和acc分別為: 0.8057 , 0.8063。
======eval finish!=========


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


'Linear_model_results_resnet18.csv'

In [57]:
csv_file_path_transposed

'layer:62_model_results_resnet18.csv'

In [ ]:
# @ unit test
from helperFunction.helperFunctions import flattenExceptDim0
output = flattenExceptDim0(torch.rand([32,512,7,7]))

output.shape

torch.Size([32, 25088])

## Generate nn output data

In [ ]:
print(test_features.shape)
print(test_labels.shape)

(50251264,)
(2003,)


### add features (LBP, csv...etc)

In [ ]:
for idx,feature in enumerate(test_features):
    feature = np.concatenate((feature , np.array(test_feature_vectors[idx])))
for idx,feature in enumerate(train_features):
    feature = np.concatenate((feature , np.array(train_feature_vectors[idx])))


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 1 dimension(s)

## XGB training

In [ ]:
import xgboost as xgb
from helperFunction.XgbHelperFunction import train_predict

# 創建XGBoost分類器
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=7)

# 訓練模型
f1, acc = train_predict(xgb_model, train_features, train_labels,  test_features, test_labels)
# xgb_model.fit(features, labels)

print(acc)

訓練 XGBClassifier 模型，樣本數: 201005056。


ValueError: Please reshape the input data into 2-dimensional matrix.

### more detail XGB info

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, log_loss

def custom_eval(preds, dtrain):
    labels = dtrain.get_label()

    # 将预测结果转换为类别标签
    mlogloss = log_loss(labels, preds, labels=np.unique(dtrain.get_label()))
    preds = np.argmax(preds, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')  # 用 'weighted' 适应类别不平衡
    return [('Accuracy', acc), ('F1-score', f1), ('mlogloss', mlogloss)]

dtrain = xgb.DMatrix(train_features, label=train_labels)
dval = xgb.DMatrix(test_features, label=test_labels)

# 设置参数，注意要更改 'objective' 为多分类的目标函数
num_class = len(np.unique(train_labels))  # 获取类别数
params = {'objective': 'multi:softprob','eval_metric': 'mlogloss', 'num_class': num_class}



evals_result = {}
bst = xgb.train(params, dtrain, num_boost_round=105, 
                evals=[(dtrain, 'train'), (dval, 'val')],
                custom_metric=custom_eval, evals_result=evals_result, 
                early_stopping_rounds=10,
                verbose_eval=True)

print(f"Best iteration: {bst.best_iteration}")


[0]	train-mlogloss:1.23198	train-Accuracy:0.87132	train-F1-score:0.86772	val-mlogloss:1.25322	val-Accuracy:0.83974	val-F1-score:0.83289


[1]	train-mlogloss:0.94329	train-Accuracy:0.87881	train-F1-score:0.87539	val-mlogloss:0.98333	val-Accuracy:0.84373	val-F1-score:0.83783
[2]	train-mlogloss:0.76241	train-Accuracy:0.88118	train-F1-score:0.87782	val-mlogloss:0.81834	val-Accuracy:0.84473	val-F1-score:0.83857
[3]	train-mlogloss:0.63912	train-Accuracy:0.88280	train-F1-score:0.87919	val-mlogloss:0.70796	val-Accuracy:0.84323	val-F1-score:0.83756
[4]	train-mlogloss:0.54987	train-Accuracy:0.88530	train-F1-score:0.88217	val-mlogloss:0.62823	val-Accuracy:0.84523	val-F1-score:0.83996
[5]	train-mlogloss:0.48412	train-Accuracy:0.88892	train-F1-score:0.88588	val-mlogloss:0.57188	val-Accuracy:0.84523	val-F1-score:0.83965
[6]	train-mlogloss:0.43472	train-Accuracy:0.88954	train-F1-score:0.88653	val-mlogloss:0.53085	val-Accuracy:0.84623	val-F1-score:0.84082
[7]	train-mlogloss:0.39629	train-Accuracy:0.89179	train-F1-score:0.88892	val-mlogloss:0.50091	val-Accuracy:0.84573	val-F1-score:0.84014
[8]	train-mlogloss:0.36640	train-Accuracy:0.8932